In [ ]:
import pandas as pd
import gymnasium as gym
import os
from ale_py import ALEInterface, roms
import time
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import cv2

env = gym.make("ALE/Frogger-v5", render_mode='rgb_array')


def reduceFrame(frame): # Make frame B&W and slim it down to reduce compute time
    gar=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    crap=gar[12:-30, 7:-7]
    resized = cv2.resize(crap, (84, 110), interpolation=cv2.INTER_AREA)
    return resized 


